# Capstone Project - DOP Spanish Restaurant in Zurich

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Selected Cuisine Venues](#cuisines)
* [Rating and Price](#ratingprice)
* [Calculations](#calculations)
* [Clustering](#clustering)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

In this project we will find an optimal location for a Spanish restaurant. differently, this work will be targeted to stakeholders interested in opening a special **Spanish restaurant in Zurich, Switzerland.**

This distinguishing feature is focused on products **labelled** by the European Union **with DOP and IGP.** DOP means Denominación de Origen Protegida - Protected Designation of Origin. IGP means Indicación Geográfica Protegida – Protected Geographical Singh. Those are marks that certify the quality and the origin of the products.

With the power of the data, we are going to **analyse the financial district** (Paradeplatz) and its surrounding areas to seek a very proper location for the restaurant. We are going to try to **compete against** the boom of **Asian cuisine and** the well-known **Italian cuisine.**

# Data <a name="data"></a>

According to the above described problem, factors to be considered:
* Number of Spanish, Asian and Italian restaurants in the zone
* Relationship between restaurants and their Quality-Price Ratio
* Distance between each other to find the best place

As data sources:
* **Foursquare API**: to get the information about the restaurants, like location, type, price or rating

# Looking for selected cuisines <a name="cuisines"></a>

####  First, we are going to find the cuisines we are focused on. We will inspect the financial district and its surrounding, thanks to its coordinates.

#### Let's import the corresponding libraries.

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# libraries to colour
import matplotlib.cm as cm
import matplotlib.colors as colors

! pip install folium==0.5.0
import folium # plotting library

# library to calculate K-means for clustering
from sklearn.cluster import KMeans

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 79 kB 6.4 MB/s  eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=8dfec543a827ed216e3df9a1ba26134126e2f957113f23b12683cb852ea076b2
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium
Folium installed
Libraries imported.


To access to **Foursquare API** we need some **credentials.**

In [2]:
CLIENT_ID = '2SV4K1J1GBRIU5UWBM2GBLCVLBOCEPVTOXCHZ0GV5WHZMYBQ' # your Foursquare ID
CLIENT_SECRET = 'D0OZPQ25JQXNFJ1KZGWALXJD22HRVH1WFRWJPGDV2K3BO2WE' # your Foursquare Secret
ACCESS_TOKEN = 'PHFJ1AHT5SFUBZMWW5MXDBNHHXYEZWBJ44CE3WS1FGO0DWZN' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 50

#### Get the coodinates of Paradeplatz with its address thanks to geopy.

In [3]:
address = 'Paradeplatz, 8001 Zürich'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Coordinates of Paradeplatz: {}, {}'.format(latitude, longitude))

Coordinates of Paradeplatz: 47.3699051, 8.5393473


#### We are going to start with the Spanish Cuisine in a radius of 1000m. 

This time we are going to use 'https,//api.foursquare,com/v2/**venues/explore** to get the information more related to location, name and category.

In [4]:
search_query = 'spanish'
radius = 1000
print(search_query + ' .... OK!')

spanish .... OK!


In [5]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    latitude, 
    longitude,
    ACCESS_TOKEN, 
    VERSION, 
    search_query, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=2SV4K1J1GBRIU5UWBM2GBLCVLBOCEPVTOXCHZ0GV5WHZMYBQ&client_secret=D0OZPQ25JQXNFJ1KZGWALXJD22HRVH1WFRWJPGDV2K3BO2WE&ll=47.3699051,8.5393473&oauth_token=PHFJ1AHT5SFUBZMWW5MXDBNHHXYEZWBJ44CE3WS1FGO0DWZN&v=20180604&query=spanish&radius=1000&limit=50'

#### Get the results of the request.

In [6]:
results = requests.get(url).json()['response']['groups'][0]['items']
#results = results['response']['groups'][0]['items']
results

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4b05888af964a520a8cc22e3',
   'name': 'Taverna Catalana',
   'location': {'address': 'Glockengasse 8',
    'lat': 47.372143352616774,
    'lng': 8.540148697935969,
    'labeledLatLngs': [{'label': 'display',
      'lat': 47.372143352616774,
      'lng': 8.540148697935969}],
    'distance': 256,
    'postalCode': '8001',
    'cc': 'CH',
    'city': 'Zürich',
    'state': 'Zürich',
    'country': 'Schweiz',
    'formattedAddress': ['Glockengasse 8', '8001 Zürich']},
   'categories': [{'id': '4bf58dd8d48988d150941735',
     'name': 'Spanish Restaurant',
     'pluralName': 'Spanish Restaurants',
     'shortName': 'Spanish',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/spanish_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},
  'referralId': 'e-0-4b05888af964a

#### This function will be used to find the category of the restaurant. In this way we will can filter some erroneous results.

In [7]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Now is time to transform the file into a data frame and filter the desired colummns.

In [8]:
# transform json file into data frame
spanish_venues  = pd.json_normalize(results)

# select the desired columms
filtered_columns_S = ['venue.name', 
                      'venue.categories',
                      'venue.location.address',
                      'venue.location.postalCode',
                      'venue.location.city',
                      'venue.location.lat', 
                      'venue.location.lng',
                      'venue.id']

spanish_venues = spanish_venues.loc[:, filtered_columns_S]

# filter the category for each row
spanish_venues['venue.categories'] = spanish_venues.apply(get_category_type, axis=1)

# clean columns
spanish_venues.columns = [col.split(".")[-1] for col in spanish_venues.columns]

spanish_venues

,name,categories,address,postalCode,city,lat,lng,id
0,Taverna Catalana,Spanish Restaurant,Glockengasse 8,8001,Zürich,47.372143,8.540149,4b05888af964a520a8cc22e3
1,Bodega Española,Spanish Restaurant,Münstergasse 15,8001,Zürich,47.371288,8.544152,4b05888af964a5208ecc22e3
2,Rechberg 1837,Spanish Restaurant,Chorgasse 20,8001,Zürich,47.374050,8.546306,4c76ad8fff1fb60c97a5f9a7
3,Tasca Romero,Spanish Restaurant,Niederdorfstr. 37,8001,Zürich,47.374905,8.543902,4bc05ca5f8219c740c77b110
4,Restaurant Emilio,Spanish Restaurant,Zweierstr. 9,8004,Zürich,47.372937,8.527220,4b058888f964a5200acc22e3
5,El Lokal,Bar,Gessnerallee 11,8001,Zürich,47.374365,8.533583,4b058888f964a5201bcc22e3
6,Wüste Bar,Bar,Oberdorfstrasse 7,8001,Zürich,47.368565,8.545591,4b8994e1f964a520974332e3
7,Restaurant Madrid,Spanish Restaurant,Froschaugasse 15,8001,Zürich,47.373241,8.544728,4b058888f964a52003cc22e3
8,Almodobar – Bar Lounge,Bar,Bleicherweg 68,8002,Zürich,47.366955,8.533310,4b5d8d51f964a5203b6129e3
9,Löweneck,Bar,Löwenstrasse 34,NaN,Zürich,47.374989,8.536324,52ed129911d227fbc549457d


#### Filtering categories and renaming the colummns.

In [9]:
Filtered_spanish_tapas_rows = ['Spanish Restaurant','Tapas Restaurant']

s_restaurants = spanish_venues.loc[spanish_venues['categories'].isin(Filtered_spanish_tapas_rows)]
s_restaurants = s_restaurants.drop_duplicates().reset_index().drop(columns = 'index')

# rename the columns 
s_restaurants = s_restaurants.rename(columns = {'name':'Restaurant Name',
                                                'categories':'Restaurant Category',
                                                'address':'Address',
                                                'postalCode':'Postal Code',
                                                'city':'City',
                                                'lat':'Latitude',
                                                'lng':'Longitude',
                                                'id':'Restaurant id',
                                                })
s_restaurants

,Restaurant Name,Restaurant Category,Address,Postal Code,City,Latitude,Longitude,Restaurant id
0,Taverna Catalana,Spanish Restaurant,Glockengasse 8,8001,Zürich,47.372143,8.540149,4b05888af964a520a8cc22e3
1,Bodega Española,Spanish Restaurant,Münstergasse 15,8001,Zürich,47.371288,8.544152,4b05888af964a5208ecc22e3
2,Rechberg 1837,Spanish Restaurant,Chorgasse 20,8001,Zürich,47.374050,8.546306,4c76ad8fff1fb60c97a5f9a7
3,Tasca Romero,Spanish Restaurant,Niederdorfstr. 37,8001,Zürich,47.374905,8.543902,4bc05ca5f8219c740c77b110
4,Restaurant Emilio,Spanish Restaurant,Zweierstr. 9,8004,Zürich,47.372937,8.527220,4b058888f964a5200acc22e3
5,Restaurant Madrid,Spanish Restaurant,Froschaugasse 15,8001,Zürich,47.373241,8.544728,4b058888f964a52003cc22e3
6,Juan Costa,Spanish Restaurant,NaN,8002,Zürich,47.365811,8.532318,4d80a0638edaa143100df239
7,Casa Viejo,Tapas Restaurant,NaN,NaN,NaN,47.373203,8.543606,59c2780cb1ec1353d74670e8
8,Sein,Tapas Restaurant,NaN,NaN,NaN,47.376246,8.540149,51adcc1e498e2abd4679e670


In [10]:
print('{} spanish restaurants venues were returned by Foursquare.'.format(s_restaurants.shape[0]))

9 spanish restaurants venues were returned by Foursquare.


#### Let's create a map of Zurich to see where the restaurants are.

In [11]:
# create map of Zurich using latitude and longitude values
map_zurich = folium.Map(location=[latitude, longitude], zoom_start=15)

# add markers to map
for lat, lng, name in zip(s_restaurants['Latitude'],
                          s_restaurants['Longitude'],
                          s_restaurants['Restaurant Name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_zurich)  
    
map_zurich

#### Now we are going to create a function to repeat the previous process but quite faster.

In [12]:
def restaurant_name_category_location(search_query_function:str, radius=1000):

    source = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        latitude, 
        longitude,
        ACCESS_TOKEN, 
        VERSION, 
        search_query_function, 
        radius, 
        LIMIT)
    
    # get data as json file
    results_function = requests.get(source).json()['response']['groups'][0]['items']
    # transform json file into data frame
    venues  = pd.json_normalize(results_function)
    
    # filter the desired colummns
    filtered_columns = ['venue.name', 
                      'venue.categories',
                      'venue.location.address',
                      'venue.location.postalCode',
                      'venue.location.city',
                      'venue.location.lat', 
                      'venue.location.lng',
                      'venue.id']
    venues = venues.loc[:, filtered_columns]

    # filter the category for each row
    venues['venue.categories'] = venues.apply(get_category_type, axis=1)

    # clean columns
    venues.columns = [col.split(".")[-1] for col in venues.columns]
    
    # change columns names
    venues = venues.rename(columns = {'name':'Restaurant Name',
                                    'categories':'Restaurant Category',
                                    'address':'Address',
                                    'postalCode':'Postal Code',
                                    'city':'City',
                                    'lat':'Latitude',
                                    'lng':'Longitude',
                                    'id':'Restaurant id',
                                    })

    return venues

#### Now is time for the other 2 cuisines. First, the Asian one. 

In this case we decided to omit the Indian cuisine. Although this is also from Asia, it does not have the same consideration as the other like sushi or Chinese, for example.

In [13]:
# call the created function
a_restaurant = restaurant_name_category_location('asian')

# desired filtered rows
filtered_a_rows = ['Thai Restaurant',
                   'Asian Restaurant',
                   'Chinese Restaurant',
                   'Vietnamese Restaurant',
                   'Japanese Restaurant',
                   'Sushi Restaurant',
                   'Korean Restaurant']

a_restaurants = a_restaurant.loc[a_restaurant['Restaurant Category'].isin(filtered_a_rows)].drop_duplicates().reset_index().drop(columns = 'index')
            
a_restaurants.head()

,Restaurant Name,Restaurant Category,Address,Postal Code,City,Latitude,Longitude,Restaurant id
0,Ban Song Thai,Thai Restaurant,Kirchgasse 6,8001,Zürich,47.369395,8.544136,4b5b5066f964a520ddf328e3
1,Tao's,Asian Restaurant,Augustinerstr.3,8001,Zürich,47.371342,8.540016,4b5ae213f964a520afd828e3
2,Lucky Dumpling,Chinese Restaurant,Sihlporte 3,8001,Zürich,47.372650,8.533830,5ac5185d535d6f0d68e91a94
3,Nagasui,Asian Restaurant,Selnaustr. 16,8001,Zürich,47.370277,8.533182,4b0f0094f964a520ee5d23e3
4,Nippon Sushi Bar,Asian Restaurant,im Jelmoli,8001,Zürich,47.374152,8.536871,4e7db9be9adffb299198c4b0


In [14]:
print('{} Asian restaurants venues were returned by Foursquare.'.format(a_restaurants.shape[0]))

41 Asian restaurants venues were returned by Foursquare.


#### Italian cuisine.

In [15]:
# call function restaurant_name_category_location
i_restaurant = restaurant_name_category_location('italian')

# desired filtered rows
filtered_i_rows = ['Italian Restaurant']

i_restaurants = i_restaurant.loc[i_restaurant['Restaurant Category'].isin(filtered_i_rows)].drop_duplicates().reset_index().drop(columns = 'index')
            
i_restaurants.head()

,Restaurant Name,Restaurant Category,Address,Postal Code,City,Latitude,Longitude,Restaurant id
0,Bindella,Italian Restaurant,In Gassen 6,8001,Zürich,47.370618,8.540610,4b081d40f964a520110423e3
1,Cantinetta Antinori,Italian Restaurant,Augustinergasse 25,8001,Zürich,47.372262,8.538558,4b698ca4f964a520e0a62be3
2,Luigia,Italian Restaurant,Talstrasse,NaN,Zürich,47.370517,8.535642,5cbb77ac811045002c7c0cb2
3,Ristorante Orsini,Italian Restaurant,Waaggasse 3,NaN,Zürich,47.369694,8.540191,4df1273645dddcd92cb8bc1e
4,Spaghetti Factory Rosenhof,Italian Restaurant,Niederdorfstr. 5,8001,Zürich,47.372952,8.543711,4b05888af964a520a3cc22e3


In [16]:
print('{} Italian restaurants venues were returned by Foursquare.'.format(i_restaurants.shape[0]))

42 Italian restaurants venues were returned by Foursquare.


#### Let's create a map to see all selected cuisines in the city of Zurich.

In [17]:
# create map of Zurich using latitude and longitude values
map_zurich = folium.Map(location=[latitude, longitude], zoom_start=15)

# Spanish
for lat, lng, name in zip(s_restaurants['Latitude'],
                          s_restaurants['Longitude'],
                          s_restaurants['Restaurant Name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_zurich) 
    
# Asian
for lat, lng, name in zip(a_restaurants['Latitude'],
                          a_restaurants['Longitude'],
                          a_restaurants['Restaurant Name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_zurich) 

# Italian
for lat, lng, name in zip(i_restaurants['Latitude'],
                          i_restaurants['Longitude'],
                          i_restaurants['Restaurant Name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_zurich) 
    
    
    
    
map_zurich

# Rating and Price <a name="ratingprice"></a>

Now we need to use 'https,//api.foursquare,com/v2/venues/**venue id/ of each restaurant** to get more concret information about the restaurant as rating or price, among others.

In [18]:
def insert_rating_price_to_restaurants(restaurants_data_frame):
    
    # list to store ids 
    restaurants_id_list = list(restaurants_data_frame['Restaurant id'])
    # lists to store price and rating
    price = []
    rating = []

    # iterate all the ids trough the api 
    for i in range(len(restaurants_data_frame)):

        venue_id = restaurants_id_list[i]

        url_price_rating = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(
            venue_id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            ACCESS_TOKEN,
            VERSION)
        
        # request
        try:
            result_price = requests.get(url_price_rating).json()['response']['venue']['price']['tier'] 
        except:
            result_price = 0
        try:# as some venues could not have rating, we give them the value of 0
            result_rating = requests.get(url_price_rating).json()['response']['venue']['rating']
        except:
            result_rating = 0
        
        # append the values to the lists
        rating.append(result_rating)
        price.append(result_price)       
    
    # insert values to restaurants data frame with some defensive programming
    try:
        restaurants_data_frame.insert(8,'Rating',rating)
    except:
        restaurants_data_frame = restaurants_data_frame  
    try:
        restaurants_data_frame.insert(9,'Price',price)
    except:
        restaurants_data_frame = restaurants_data_frame
    
    return restaurants_data_frame

#### Get the new dataframe for Spanish restaurants.

In [19]:
s_restaurants_final = insert_rating_price_to_restaurants(s_restaurants)
s_restaurants_final.head()

,Restaurant Name,Restaurant Category,Address,Postal Code,City,Latitude,Longitude,Restaurant id,Rating,Price
0,Taverna Catalana,Spanish Restaurant,Glockengasse 8,8001,Zürich,47.372143,8.540149,4b05888af964a520a8cc22e3,7.6,2
1,Bodega Española,Spanish Restaurant,Münstergasse 15,8001,Zürich,47.371288,8.544152,4b05888af964a5208ecc22e3,7.3,2
2,Rechberg 1837,Spanish Restaurant,Chorgasse 20,8001,Zürich,47.374050,8.546306,4c76ad8fff1fb60c97a5f9a7,8.3,2
3,Tasca Romero,Spanish Restaurant,Niederdorfstr. 37,8001,Zürich,47.374905,8.543902,4bc05ca5f8219c740c77b110,7.2,2
4,Restaurant Emilio,Spanish Restaurant,Zweierstr. 9,8004,Zürich,47.372937,8.527220,4b058888f964a5200acc22e3,7.7,2


#### Asian restaurants.

In [20]:
a_restaurants_final = insert_rating_price_to_restaurants(a_restaurants)
a_restaurants_final.head()

,Restaurant Name,Restaurant Category,Address,Postal Code,City,Latitude,Longitude,Restaurant id,Rating,Price
0,Ban Song Thai,Thai Restaurant,Kirchgasse 6,8001,Zürich,47.369395,8.544136,4b5b5066f964a520ddf328e3,8.7,2
1,Tao's,Asian Restaurant,Augustinerstr.3,8001,Zürich,47.371342,8.540016,4b5ae213f964a520afd828e3,7.9,3
2,Lucky Dumpling,Chinese Restaurant,Sihlporte 3,8001,Zürich,47.372650,8.533830,5ac5185d535d6f0d68e91a94,8.5,1
3,Nagasui,Asian Restaurant,Selnaustr. 16,8001,Zürich,47.370277,8.533182,4b0f0094f964a520ee5d23e3,7.7,2
4,Nippon Sushi Bar,Asian Restaurant,im Jelmoli,8001,Zürich,47.374152,8.536871,4e7db9be9adffb299198c4b0,8.6,2


#### Italian restaurants.

In [21]:
i_restaurants_final = insert_rating_price_to_restaurants(i_restaurants)
i_restaurants_final.head()

,Restaurant Name,Restaurant Category,Address,Postal Code,City,Latitude,Longitude,Restaurant id,Rating,Price
0,Bindella,Italian Restaurant,In Gassen 6,8001,Zürich,47.370618,8.540610,4b081d40f964a520110423e3,8.5,2
1,Cantinetta Antinori,Italian Restaurant,Augustinergasse 25,8001,Zürich,47.372262,8.538558,4b698ca4f964a520e0a62be3,8.5,2
2,Luigia,Italian Restaurant,Talstrasse,NaN,Zürich,47.370517,8.535642,5cbb77ac811045002c7c0cb2,8.0,2
3,Ristorante Orsini,Italian Restaurant,Waaggasse 3,NaN,Zürich,47.369694,8.540191,4df1273645dddcd92cb8bc1e,7.5,2
4,Spaghetti Factory Rosenhof,Italian Restaurant,Niederdorfstr. 5,8001,Zürich,47.372952,8.543711,4b05888af964a520a3cc22e3,7.9,2


#### Some restaurants do not have a price range or a rating. We will remove them to calculate a proper quality-price ratio.

In [22]:
s_restaurants_final = s_restaurants_final[s_restaurants_final.Rating != 0.0]
s_restaurants_final = s_restaurants_final[s_restaurants_final.Price != 0.0]
a_restaurants_final = a_restaurants_final[a_restaurants_final.Rating != 0.0]
a_restaurants_final = a_restaurants_final[a_restaurants_final.Price != 0.0]
i_restaurants_final = i_restaurants_final[i_restaurants_final.Rating != 0.0]
i_restaurants_final = i_restaurants_final[i_restaurants_final.Price != 0.0]

print('The final amount of Spanish cuisine is', len(s_restaurants_final))
print('The final amount of Asian cuisine is', len(a_restaurants_final))
print('The final amount of Italian cuisine is', len(i_restaurants_final))

The final amount of Spanish cuisine is 7
The final amount of Asian cuisine is 41
The final amount of Italian cuisine is 40


#### Let's put the quality-price ratio into the final dataframes.

In [23]:
ratio = ((s_restaurants_final['Rating']*10) /s_restaurants_final['Price']).astype(int)
ratio = list(ratio)
try:
    s_restaurants_final.insert(10,'Ratio',ratio)
except:
    s_restaurants_final = s_restaurants_final
s_restaurants_final

,Restaurant Name,Restaurant Category,Address,Postal Code,City,Latitude,Longitude,Restaurant id,Rating,Price,Ratio
0,Taverna Catalana,Spanish Restaurant,Glockengasse 8,8001,Zürich,47.372143,8.540149,4b05888af964a520a8cc22e3,7.6,2,38
1,Bodega Española,Spanish Restaurant,Münstergasse 15,8001,Zürich,47.371288,8.544152,4b05888af964a5208ecc22e3,7.3,2,36
2,Rechberg 1837,Spanish Restaurant,Chorgasse 20,8001,Zürich,47.374050,8.546306,4c76ad8fff1fb60c97a5f9a7,8.3,2,41
3,Tasca Romero,Spanish Restaurant,Niederdorfstr. 37,8001,Zürich,47.374905,8.543902,4bc05ca5f8219c740c77b110,7.2,2,36
4,Restaurant Emilio,Spanish Restaurant,Zweierstr. 9,8004,Zürich,47.372937,8.527220,4b058888f964a5200acc22e3,7.7,2,38
5,Restaurant Madrid,Spanish Restaurant,Froschaugasse 15,8001,Zürich,47.373241,8.544728,4b058888f964a52003cc22e3,6.7,2,33
6,Juan Costa,Spanish Restaurant,NaN,8002,Zürich,47.365811,8.532318,4d80a0638edaa143100df239,5.6,2,28


In [24]:
ratio = ((a_restaurants_final['Rating']*10) /a_restaurants_final['Price']).astype(int)
ratio = list(ratio)
try:
    a_restaurants_final.insert(10,'Ratio',ratio)
except:
    a_restaurants_final = a_restaurants_final
a_restaurants_final.head()

,Restaurant Name,Restaurant Category,Address,Postal Code,City,Latitude,Longitude,Restaurant id,Rating,Price,Ratio
0,Ban Song Thai,Thai Restaurant,Kirchgasse 6,8001,Zürich,47.369395,8.544136,4b5b5066f964a520ddf328e3,8.7,2,43
1,Tao's,Asian Restaurant,Augustinerstr.3,8001,Zürich,47.371342,8.540016,4b5ae213f964a520afd828e3,7.9,3,26
2,Lucky Dumpling,Chinese Restaurant,Sihlporte 3,8001,Zürich,47.372650,8.533830,5ac5185d535d6f0d68e91a94,8.5,1,85
3,Nagasui,Asian Restaurant,Selnaustr. 16,8001,Zürich,47.370277,8.533182,4b0f0094f964a520ee5d23e3,7.7,2,38
4,Nippon Sushi Bar,Asian Restaurant,im Jelmoli,8001,Zürich,47.374152,8.536871,4e7db9be9adffb299198c4b0,8.6,2,43


In [25]:
ratio = ((i_restaurants_final['Rating']*10) /i_restaurants_final['Price']).astype(int)
ratio = list(ratio)
try:
    i_restaurants_final.insert(10,'Ratio',ratio)
except:
    i_restaurants_final = i_restaurants_final
i_restaurants_final.head()

,Restaurant Name,Restaurant Category,Address,Postal Code,City,Latitude,Longitude,Restaurant id,Rating,Price,Ratio
0,Bindella,Italian Restaurant,In Gassen 6,8001,Zürich,47.370618,8.540610,4b081d40f964a520110423e3,8.5,2,42
1,Cantinetta Antinori,Italian Restaurant,Augustinergasse 25,8001,Zürich,47.372262,8.538558,4b698ca4f964a520e0a62be3,8.5,2,42
2,Luigia,Italian Restaurant,Talstrasse,NaN,Zürich,47.370517,8.535642,5cbb77ac811045002c7c0cb2,8.0,2,40
3,Ristorante Orsini,Italian Restaurant,Waaggasse 3,NaN,Zürich,47.369694,8.540191,4df1273645dddcd92cb8bc1e,7.5,2,37
4,Spaghetti Factory Rosenhof,Italian Restaurant,Niederdorfstr. 5,8001,Zürich,47.372952,8.543711,4b05888af964a520a3cc22e3,7.9,2,39


# Calculations <a name="calculations"></a>

#### Amount of Spanish restaurants compared to Asian and Italian.

In [26]:
total_selected_cuisines = len(s_restaurants_final) + len(a_restaurants_final) + len(i_restaurants_final)
percentage_s_restaurants = (len(s_restaurants_final) / total_selected_cuisines) *100
print('Total restaurants of selected cuisines:',(total_selected_cuisines))
print('Percentage of Spanish restaurants: {:.2f}%'.format(len(s_restaurants_final) / total_selected_cuisines * 100))
print('Percentage of Spanish restaurants per Asian restaurant: {:.2f}%'.format(len(s_restaurants_final) / len(a_restaurants_final) * 100))
print('Percentage of Spanish restaurants per Italian restaurant: {:.2f}%'.format(len(s_restaurants_final) / len(i_restaurants_final) * 100))

Total restaurants of selected cuisines: 88
Percentage of Spanish restaurants: 7.95%
Percentage of Spanish restaurants per Asian restaurant: 17.07%
Percentage of Spanish restaurants per Italian restaurant: 17.50%


#### Relatioship between the quality-price ratio compared to the other restaurants.

In [27]:
print('Average quality-price ratio of Spanish restaurants: {:.2f}%'.format(s_restaurants_final['Ratio'].sum() / len(s_restaurants_final)))
print('Average quality-price ratio of Asian restaurants: {:.2f}%'.format(a_restaurants_final['Ratio'].sum() / len(a_restaurants_final)))
print('Average quality-price ratio of Italian restaurants: {:.2f}%'.format(i_restaurants_final['Ratio'].sum() / len(i_restaurants_final)))

Average quality-price ratio of Spanish restaurants: 35.71%
Average quality-price ratio of Asian restaurants: 41.73%
Average quality-price ratio of Italian restaurants: 37.38%


# Clustering <a name="clustering"></a>

#### Creating a dataframe with all the 86 restaurants.

In [28]:
total_restaurants = s_restaurants_final.append([a_restaurants_final,i_restaurants_final]).reset_index().drop(columns = ['index'])
total_restaurants

,Restaurant Name,Restaurant Category,Address,Postal Code,City,Latitude,Longitude,Restaurant id,Rating,Price,Ratio
0,Taverna Catalana,Spanish Restaurant,Glockengasse 8,8001,Zürich,47.372143,8.540149,4b05888af964a520a8cc22e3,7.6,2,38
1,Bodega Española,Spanish Restaurant,Münstergasse 15,8001,Zürich,47.371288,8.544152,4b05888af964a5208ecc22e3,7.3,2,36
2,Rechberg 1837,Spanish Restaurant,Chorgasse 20,8001,Zürich,47.374050,8.546306,4c76ad8fff1fb60c97a5f9a7,8.3,2,41
3,Tasca Romero,Spanish Restaurant,Niederdorfstr. 37,8001,Zürich,47.374905,8.543902,4bc05ca5f8219c740c77b110,7.2,2,36
4,Restaurant Emilio,Spanish Restaurant,Zweierstr. 9,8004,Zürich,47.372937,8.527220,4b058888f964a5200acc22e3,7.7,2,38
...,...,...,...,...,...,...,...,...,...,...,...
83,Antica Roma,Italian Restaurant,NaN,NaN,Zürich,47.370305,8.531740,5331c36b498ed77d72ae3612,6.1,2,30
84,Pasta Station,Italian Restaurant,NaN,NaN,Zürich,47.373669,8.530009,4e0ceddf1f6e9300e4eb3b75,6.2,2,31
85,Vallocaia Ristorante,Italian Restaurant,NaN,NaN,Zürich,47.373424,8.543809,4eb06d6b722e61d2b4d8d02b,5.9,2,29
86,Dal Nastro - Bar del Gusto,Italian Restaurant,Sihlporte 3,8001,Zürich,47.372747,8.533913,52a72d82498efa07113d8047,5.7,2,28


#### Time to cluster the restaurants with the desired amount, thanks to K-means.

In [29]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(total_restaurants[['Latitude','Longitude']])

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 3, 1, 0, 2, 1, 0], dtype=int32)

#### With K-means we can also just calculate the centroide of all restaurants to find the best posible location compared to the others.

In [30]:
# set number of clusters
kclusters_centroide = 1

# run k-means clustering
kmeans_centroide = KMeans(n_clusters=kclusters_centroide, random_state=0).fit(total_restaurants[['Latitude','Longitude']])

# generate the centroides and put into a variable
clusters_center = kmeans_centroide.cluster_centers_

clusters_center

array([[47.37218021,  8.53831783]])

##### And finally let's plot all the restaurants on a map of Zurich and get some information just clicking on the marker.

In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=15)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Centroide of all selected venue
label_centroide = 'According to cetroide of K-means, this is the optimal location of the restaurant'
folium.CircleMarker(
    [clusters_center[0][0],clusters_center[0][1]],
    radius=10,
    popup=label_centroide,
    color='red',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7).add_to(map_clusters)

# add markers to the map
markers_colors = []
for lat, lng, name, address, postalcode, city, price, rating, cluster in zip(total_restaurants['Latitude'],
                                                                            total_restaurants['Longitude'],
                                                                            total_restaurants['Restaurant Name'],
                                                                            total_restaurants['Address'],
                                                                            total_restaurants['Postal Code'],
                                                                            total_restaurants['City'],
                                                                            total_restaurants['Price'],
                                                                            total_restaurants['Rating'],
                                                                            kmeans.labels_):
    label = '{}. {}, {} {}. Price-Range (1-4): {} Rating: {}'.format(name, address, postalcode, city, price, rating)
    label = folium.Popup(label, max_width=700, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=2).add_to(map_clusters)


       
map_clusters

# Conclusion <a name="conclusion"></a>

We have found a privileged niche market. the number of Spanish restaurants is derisory compared to the other selected cuisines. The quality-price ratio is as well the lowest one. Why is that happening, although Spain is the 4th restaurant with more Michelin Stars of the world? 234 in particular for this 2021. 

According to the centroide based on all selected restaurants there is a great lack of Spanish ones. All this, added to the high quality we can offer with our certified products, we consider that open a Spanish restaurant in the center of Zurich is a very good investment.